In [1]:
import pandas as pd
import os
import numpy as np
import time
import wandb
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle

from matplotlib import image
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader

from cfgs.on_track_cls import OnTrackClsCfg
from dp_util.early_stopping import EarlyStopper
from models.on_track_cls import OnTrackClsNet
from dp_spec.img import preprocess_img
from dp_util.dataset import CustomDataset
from dp_util.training import valid

C:\Users\auyin11\anaconda3\envs\rl_game_pip\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\auyin11\anaconda3\envs\rl_game_pip\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir_auto_img = r'D:\file\data\racing\auto_img'

cfg = OnTrackClsCfg(test_mode=False, dir_data='./')

wandb.init(project=cfg.project, entity=cfg.entity,
           group=f'{cfg.user}_{cfg.model}', job_type="train",
           name= cfg.version)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: auyin11 (kaggle_winner). Use `wandb login --relogin` to force relogin


device: cuda


In [3]:
list_path = []
list_lbl = []

for file in os.listdir(os.path.join(dir_auto_img, 'on')):
    list_path.append(os.path.join(dir_auto_img, 'on', file))
    list_lbl.append(['on'])

for file in os.listdir(os.path.join(dir_auto_img, 'off')):
    list_path.append(os.path.join(dir_auto_img, 'off', file))
    list_lbl.append(['off'])
    
encoder = OneHotEncoder(handle_unknown='ignore')
list_lbl_array = encoder.fit_transform(list_lbl).toarray()


path_train, path_test, y_train, y_test = train_test_split(list_path, list_lbl_array, test_size=0.2, stratify=list_lbl, shuffle=True) 
path_train, path_valid, y_train, y_valid = train_test_split(path_train, y_train, test_size=0.2, stratify=y_train, shuffle=True) 

ts_y_train = torch.tensor(y_train, device=device)
ts_y_valid = torch.tensor(y_valid, device=device)
ts_y_test = torch.tensor(y_test, device=device)

list_x_y = []
for x, y in  zip([torch.tensor(preprocess_img(image.imread(x)),
                               dtype=torch.float32) for x in path_train], ts_y_train):
    list_x_y.append((x, y))
train_loader = torch.utils.data.DataLoader(dataset=CustomDataset(list_x_y),
                                           batch_size=cfg.BATCH_SIZE,
                                           shuffle=False)
list_x_y = []
for x, y in  zip([torch.tensor(preprocess_img(image.imread(x)),
                               dtype=torch.float32) for x in path_valid], ts_y_valid):
    list_x_y.append((x, y))
valid_loader = torch.utils.data.DataLoader(dataset=CustomDataset(list_x_y),
                                           batch_size=cfg.BATCH_SIZE,
                                           shuffle=False)

list_x_y = []
for x, y in  zip([torch.tensor(preprocess_img(image.imread(x)),
                               dtype=torch.float32) for x in path_test], ts_y_test):
    list_x_y.append((x, y))
test_loader = torch.utils.data.DataLoader(dataset=CustomDataset(list_x_y),
                                           batch_size=cfg.BATCH_SIZE,
                                           shuffle=False)

In [4]:
net = OnTrackClsNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=cfg.LR)

es = EarlyStopper(cfg.md_name, smaller_is_better=True, patience=30, sf=cfg.SF)

In [5]:
for epoch in range(500):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    
    for data in train_loader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        inputs = torch.unsqueeze(inputs, dim=1)
        
        total += labels.size(0)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    if not es.early_stop:
        
        train_avg_loss = running_loss/total
        valid_avg_loss, _ = valid(net, valid_loader, criterion,
                                  device, verbose=False)
        wandb.log({'train_avg_loss': train_avg_loss, 'valid_avg_loss': valid_avg_loss})
        
        es(valid_avg_loss, net)
    else:
        break

the value is started at 0.02260
the value is improved from 0.02107 to 0.01760 (best_epoch: 5)
the value is improved from 0.00561 to 0.00514 (best_epoch: 10)
the value is improved from 0.00448 to 0.00444 (best_epoch: 15)
the value is improved from 0.00402 to 0.00399 (best_epoch: 20)
the value is improved from 0.00342 to 0.00336 (best_epoch: 25)
the value is improved from 0.00305 to 0.00292 (best_epoch: 30)
the value is improved from 0.00264 to 0.00259 (best_epoch: 35)
the value is improved from 0.00240 to 0.00237 (best_epoch: 40)
the value is improved from 0.00217 to 0.00214 (best_epoch: 45)
the value is improved from 0.00203 to 0.00191 (best_epoch: 50)

Early stop at best_epoch = 51 and best_value = 0.00180


In [7]:
test_avg_loss, _ = valid(net, test_loader, criterion, device)
wandb.log({'test_avg_loss': test_avg_loss})

Accuracy of the network on the 190 images: 98 % ang avg_loss: 0.00426


In [8]:
with open('ont_hot_encoder.pkl','wb') as output:
      pickle.dump(encoder, output)

In [9]:
cfg.md_name

'on_track_classifier_v_2_0_5.pth'